In [ ]:
prefix = 'kmeans'

import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [ ]:
import matplotlib.pyplot as plt
df = pd.read_csv('training-data/kmeans_input.csv')
x = df.iloc[:,0].values
y = df.iloc[:,1].values
plt.scatter(x,y)

In [ ]:
WORK_DIRECTORY = 'training-data'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest'

kmeans_estimator = sage.estimator.Estimator(
    image, 
    role, 
    1, 
    'ml.c4.2xlarge', 
    output_path="s3://{}/{}/kmeans_output".format(sess.default_bucket(), prefix),
    sagemaker_session=sess)

kmeans_estimator.fit(data_location)

In [ ]:
import tarfile
import boto3
from io import BytesIO
import pickle

s3 = boto3.resource('s3')
path = "{}/kmeans_output/{}/output/model.tar.gz".format(prefix, kmeans_estimator.latest_training_job.job_name)
bucket = s3.Bucket(sess.default_bucket())
compressed = BytesIO()
bucket.download_fileobj(path, compressed)
compressed.seek(0)
tar = tarfile.open("r:gz", fileobj=compressed)
f = tar.extractfile('kmeans.sav')
m = pickle.load(f, encoding='latin1')

In [ ]:
plt.scatter(x,y,c=m.labels_)

In [ ]:
from sagemaker.predictor import csv_serializer
kmeans_predictor = kmeans_estimator.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

In [ ]:
input = """x1,x2
-10,-10"""
r = kmeans_pred]\\ictor.predict(input).decode('utf-8')
print(r)